<b><font color="black" size="+3">PULSE from image url</font></b>

Given a full size image url, this notebook first crop and produce a low-resolution facial image, and invoke [PULSE](https://github.com/adamian98/pulse) algoirthm (using StyleGAN) to generate a high-resolution image that are perceptually realistic. Then you can compare how well the algoirthm does.

git repository: https://github.com/ctawong/PULSE_from_image_url


In [ ]:
import torch
import torchvision
from pathlib import Path
import shutil
import os

if not Path("PULSE.py").exists():
  if Path("pulse").exists():
    %cd /content/pulse
  else:
    !git clone https://github.com/ctawong/pulse
    %cd /content/pulse
    !mkdir input/
    toPIL = torchvision.transforms.ToPILImage()
    toTensor = torchvision.transforms.ToTensor()
    from bicubic import BicubicDownSample
    D = BicubicDownSample(factor=1)




from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from PULSE import PULSE
from google.colab import files
from bicubic import BicubicDownSample
from IPython import display
from IPython.display import display
from IPython.display import clear_output
import numpy as np
from drive import open_url
from mpl_toolkits.axes_grid1 import ImageGrid
import requests
%matplotlib inline

#@markdown <font color="red" size="+1">←</font><font color="black" size="+1"> Replace **Photo_URL** below and click play button on left to start! </font>


#@markdown ---
#@markdown ## Required settings:
Photo_URL = "https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTsjHNnAwsbqz0njBgk4IbPTjl1AcTA_ivbRQ&usqp=CAU" #@param {type:"string"}
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

doDownload = True
try:
  if last_Photo_URL == Photo_URL:
    doDownload = False
    print("Same Photo_URL. Skip download.")
except:
  pass
if doDownload:
  print("Download photo from %s"%Photo_URL)
  # remove previous content
  if Path("realpics").exists():
    shutil.rmtree('realpics')
  !mkdir realpics
  if Path("input").exists():
    shutil.rmtree('input')
  !mkdir input  
  r = requests.get(Photo_URL, allow_redirects=True, headers=headers)
  ext=r.headers.get('content-type').split('/')[1]
  realPicName = 'realpics/face.'+ext
  open(realPicName, 'wb').write(r.content)
  !python align_face.py
  !mv input/face_0.png input/face.png
  last_Photo_URL = Photo_URL

#@markdown ## Advanced settings:
#@markdown ##### If it cannot find solution or you want it to be more accurate,  modify the following *DEFAULT* parameters. 
#@markdown Increase `allowed_error` or `number_of_steps` to find solutions. Allowed error corresponds to L2 value.
#@markdown Change `seed` to generate a different face.

allowed_error = 0.003 #@param {type:"slider", min:0.001, max:0.006, step:0.0005}
number_of_steps = 200 #@param {type:"slider", min:100, max:1000, step:50}
seed = 171 #@param {type:"integer"}
noise_type = 'trainable'  #@param ['zero', 'fixed', 'trainable']
optimizer = 'adam'  #@param ['sgd', 'adam','sgdm', 'adamax']
learning_rate = 0.2 #@param {type:"slider", min:0, max:1, step:0.05}
learning_rate_schedule = 'linear1cycledrop'  #@param ['fixed', 'linear1cycle', 'linear1cycledrop']
clear_output()

seed = abs(seed)
try:
  os.remove("/content/pulse/runs/face.png")
except:
  pass
!python run.py \
  -eps $allowed_error\
  -seed $seed \
  -noise_type $noise_type \
  -opt_name $optimizer \
  -learning_rate $learning_rate \
  -steps $number_of_steps \
  -lr_schedule $learning_rate_schedule

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(mpimg.imread('/content/pulse/input/face.png'))
ax1.set_title('Original')

if Path('/content/pulse/runs/face.png').exists():
  ax2.imshow(mpimg.imread('/content/pulse/runs/face.png'))
  ax2.set_title('Result')
else:
  print("Error: Try increasing allowed_error and/or number_of_steps")
plt.show()

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Download result</font></b>
try: files.download('/content/pulse/runs/face.png')
except: raise Exception("No result image")